In [17]:
import numpy
import numpy as np
import math
import pandas as pd
from gt_design import *

#### Given Values

In [11]:
mechanical_eff = 0.99
gamma_air = 1.4
gamma_g = 1.33333
c_p_air = 1.005
c_p_gas = 1.148
R = 0.287

m_cool_vane_hpt = 0.1450776
m_cool_disc_hpt = 0.0797

# TURBINE INLET
T_01 = 1245.320         # [k]
P_01 = 1182.073         # [kPa]
m_dot_1 = 4.835922
M_1 = 0.125

# BETWEEN STATOR AND ROTOR
m_dot_2 = m_dot_1 + m_cool_vane_hpt

# TURBINE EXIT
T_03 = 1053.051
P_03 = 524.594
m_dot_3 = m_dot_2 + m_cool_disc_hpt

# BLADE PARAMETERS
# VANE
AR_vane = 0.5
TE_vane = 1.27/1000     # minimum trailing edge thickness in [m]
# BLADE
AR_rotor = 1.3
TE_rotor = 0.762/1000   # minimum trailing edge thickness in [m]
tip_clearance = 0.01    # minimum tip clearance span -> maximum is 0.02

## Define Ranges

Define the range for the stage loading coefficient, flow coefficient and the reaction. Using the obtained values, calculate the other values.

Use the following data:
1. Reaction ($\Lambda$) -> 0.35 - 0.65
2. Flow Coefficient ($\phi$) -> 0.6 - 1.0
3. Structural Limit ($AN^2$) -> 10000000 - 22580600 (checking for a wide range)
4. Zweifel for vane -> 0.75 - 0.90
5. Zweifel for rotor -> 0.80 - 0.95

In [12]:
range_degree_reaction = numpy.linspace(0.35, 0.65, 10)
range_stage_loading = numpy.linspace(3, 5, 10)
range_alpha_3 = numpy.linspace(10, 45, 10)
range_AN_squared = numpy.linspace(15000000, 22580600, 3)
range_zweifel_vane = numpy.linspace(0.75, 0.90, 15)
range_zweifel_rotor = numpy.linspace(0.80, 0.95, 15)
range_mach_exit = numpy.linspace(0.3, 0.55, 12)

## int main(){}

In [13]:
data = []

for i in range_mach_exit:
    for j in range_stage_loading:
        for k in range_alpha_3:
            for l in range_degree_reaction:
                for m in range_AN_squared:
                    #for n in range_zweifel_vane:
                    T_1, P_1, rho_1, C_1 = aeroturbine.calc_properties(M_1, T_01, P_01)
                    C_a_1 = C_1 * np.cos(np.radians(-10))
                    T_3, P_3, rho_3, C_3 = aeroturbine.calc_properties(i, T_03, P_03)

                    # Calculate U from the stage loading.
                    U = aeroturbine.calc_U(j)
                    C_a_3, C_w_3,  V_3, V_w_3, flow_coefficient_3, beta_3, a_3, M_3_rel, A_3 = aeroturbine.calc_stage_3(U, k, C_3, T_3, rho_3)
                    T_2, P_2, rho_2, A_2, C_a_2, flow_coefficient_2, a_2, V_w_2, beta_2, V_2, C_w_2, C_2, alpha_2, M_2, M_2_rel = aeroturbine.calc_stage_2(U, l, T_1, T_3, P_3, A_3, V_w_3)
                    
                    A_1 = m_dot_1/(rho_1 * C_a_1)
                    N, omega, r_hub, r_tip, r_meanline, h = aerostructural.calc_structural(m, A_2, U)

                    alpha_2_hub, alpha_3_hub, beta_2_hub, beta_3_hub, U_hub, V_2_hub, C_2_hub, M_2_rel_hub, M_2_hub = aeroturbine.calc_hub_angles(r_meanline, r_hub, alpha_2, k, flow_coefficient_2, flow_coefficient_3, U, C_a_2, a_2)
                    alpha_2_tip, alpha_3_tip, beta_2_tip, beta_3_tip, U_tip, V_2_tip, C_2_tip, M_2_rel_tip, M_2_tip = aeroturbine.calc_tip_angles(r_meanline, r_tip, alpha_2, k, flow_coefficient_2, flow_coefficient_3, U, C_a_2, a_2)
                    reaction_hub, reaction_tip = aeroturbine.calc_tip_hub_reaction(C_a_3, C_a_2 ,beta_2_hub, beta_2_tip, beta_3_hub, beta_3_tip, U_hub, U_tip)   

                    if V_3 > V_2 and C_2 > C_1 and rho_3 < rho_2 < rho_1: # Velocity check, density check and omega check _>
                        # Other value checks from the design requirements _>
                        if 144 <= C_3 <= 339 and 190 <= C_a_3 <= 202 and 40 <= alpha_2 <= 75 and reaction_hub > 0 and M_2_rel_hub > M_2_rel_tip and 3076 < omega < 4754:
                            if 0 < alpha_2_tip < alpha_2 < alpha_2_hub and 0 < beta_2_tip < beta_2 < beta_2_hub and 0 < alpha_3_tip < k < alpha_3_hub and beta_3_tip > beta_3 > beta_3_hub > 0:

                                # OFF DESIGN CALCULATIONS
                                rho_3_od, T_3_od, P_3_od, alpha_3_od, flow_coeff_2_od, incidence_2, v_2_od, alpha_3_rel_od, C_w_3_od, C_a_3_od, U_mean_od, flow_coeff_3_od = off_design.calc_off_design(omega, r_meanline, C_a_2, C_w_2, beta_2, beta_3,A_3)

                                if flow_coeff_3_od > flow_coeff_2_od:

                                    # CALCULATE LOSSES
                                    K_p, pitch_chord_ratio, K_accel, stagger_angle, pitch_axial_chord_ratio =  aerodynamic_losses.profile_losses.calc_K_p(M_2_rel, M_3_rel, M_2_rel_hub, P_2, P_3, r_tip, r_hub,  beta_2, beta_3, 0.80)

                                    # Calculations for throat opening
                                    c_true = (h)/AR_rotor
                                    c_a = (h * np.cos(np.radians(stagger_angle)))/AR_rotor
                                    N_rotor = math.floor((2 * np.pi * r_meanline) /(pitch_axial_chord_ratio * c_a))
                                    o_rotor = (pitch_axial_chord_ratio * c_a) * np.sin(np.radians(beta_3))
                                    K_s = aerodynamic_losses.secondary_losses.calc_K_s(K_accel, AR_rotor, beta_2, beta_3)
                                    K_TET, r_to_o, f = aerodynamic_losses.trailing_edge_losses.K_TET(o_rotor, M_2_rel, beta_2, beta_3)
                                    K = K_p + K_s + K_TET



                                    data.append((j, l, T_1, P_1, rho_1, A_1, C_1, C_a_1, i, T_3, P_3, rho_3, C_3, k, U,
                                                C_a_3, C_w_3,  V_3, V_w_3, flow_coefficient_3, beta_3, a_3, M_3_rel, A_3,
                                                T_2, P_2, rho_2, A_2, C_a_2, flow_coefficient_2, a_2, V_w_2, beta_2, V_2, C_w_2, C_2, alpha_2, M_2, M_2_rel,
                                                N, omega, r_hub, r_tip, r_meanline, h,
                                                alpha_2_hub, alpha_3_hub, beta_2_hub, beta_3_hub, U_hub, V_2_hub, C_2_hub, M_2_rel_hub, M_2_hub,
                                                alpha_2_tip, alpha_3_tip, beta_2_tip, beta_3_tip, U_tip, V_2_tip, C_2_tip, M_2_rel_tip, M_2_tip,
                                                reaction_hub, reaction_tip, N_rotor,
                                                K_p,  pitch_chord_ratio, K_accel, o_rotor,c_a,c_true,
                                                K_s, K_TET,r_to_o, f,
                                                K,
                                                rho_3_od, T_3_od, P_3_od, alpha_3_od, flow_coeff_2_od, incidence_2, v_2_od, alpha_3_rel_od, C_w_3_od, C_a_3_od, U_mean_od, flow_coeff_3_od))

                    else:
                        print("Check your code!!")
                        break


df = pd.DataFrame(data, columns = ['stage_loading','reaction_mean','T_1', 'P_1', 'rho_1', 'A_1','C_1', 'C_a_1','M_3','T_3','P_3', 'rho_3', 'C_3', 'alpha_3','U',
                                   'C_a_3', 'C_w_3',  'V_3', 'V_w_3', 'flow_coefficient_3', 'beta_3', 'a_3', 'M_3_rel', 'A_3',
                                   'T_2', 'P_2', 'rho_2', 'A_2', 'C_a_2', 'flow_coefficient_2', 'a_2', 'V_w_2', 'beta_2', 'V_2', 'C_w_2', 'C_2', 'alpha_2', 'M_2', 'M_2_rel',
                                   'N', 'omega', 'r_hub', 'r_tip', 'r_meanline', 'h',
                                   'alpha_2_hub', 'alpha_3_hub', 'beta_2_hub', 'beta_3_hub', 'U_hub', 'V_3_hub', 'C_2_hub', 'M_2_rel_hub', 'M_2_hub',
                                   'alpha_2_tip', 'alpha_3_tip', 'beta_2_tip', 'beta_3_tip', 'U_tip', 'V_3_tip', 'C_2_tip', 'M_2_rel_tip', 'M_2_tip',
                                   'reaction_hub', 'reaction_tip', 'N_rotor',
                                   'K_p',  'pitch_chord_ratio', 'K_accel', 'o_rotor', 'c_a', 'c_true',
                                   'K_s', 'K_TET', 'r_to_o', 'f' ,
                                   'K',
                                   'rho_3_od', 'T_3_od', 'P_3_od', 'alpha_3_od', 'flow_coeff_2_od', 'incidence_2', 'v_2_od', 'alpha_3_rel_od', 'C_w_3_od', 'C_a_3_od', 'U_mean_od', 'flow_coeff_3_od'])


Splitting the main dataframe

In [15]:
df_losses_meanline = df[['K_p',  'pitch_chord_ratio', 'K_accel', 'o_rotor', 'c_a', 'c_true','K_s', 'K_TET', 'r_to_o', 'f' ,'K']]
df_off_design = df[['rho_3_od', 'T_3_od', 'P_3_od', 'alpha_3_od', 'flow_coeff_2_od', 'incidence_2', 'v_2_od', 'alpha_3_rel_od', 'C_w_3_od', 'C_a_3_od', 'U_mean_od', 'flow_coeff_3_od']]

Write results to file

In [16]:
df.to_csv('_outputs/output.csv', index=False)
df_losses_meanline.to_csv('_outputs/output_losses_meanline.csv', index=False)
df_off_design.to_csv('_outputs/output_off_design.csv', index=False)